# Joining Two Vector Products

In [ ]:
import os

os.environ["VECTOR_API_HOST"] = "http://127.0.0.1:8000"
import ipyleaflet
import requests
import json
import pandas as pd
import geopandas as gpd
import descarteslabs as dl
from client.descarteslabs.vector import Table, TableOptions, models, properties as p
from shapely.geometry import Point
from pydantic import Field

## Downloading country boundaries

Download country boundarys as a GeoJSON FeatureCollection and convert the FeatureCollection to a GeoPandas dataframe.

In [ ]:
# download feature collection of country boundaries
url = "https://raw.githubusercontent.com/martynafford/natural-earth-geojson/master/110m/cultural/ne_110m_admin_0_countries.json"
response = requests.get(url)
feature_collection = response.json()

# convert feature collection to GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(feature_collection["features"], crs="EPSG:4326")

Since two Vector products are required for joining, we are going to create two Vector products from the country boundary data. We will create one Vector product containing the country name, continent, and boundary (polygon). The other Vector product will contain country name, population estimate, GDP estimate, and country centroid (point).

In [ ]:
# create dataframe for the boundary table
boundary_gdf = gdf.drop(
    gdf.columns.difference(["geometry", "NAME", "CONTINENT"]),
    axis=1,
)

# create dataframe for the population table
population_gdf = gdf.drop(
    gdf.columns.difference(["geometry", "NAME", "POP_EST", "GDP_MD_EST"]),
    axis=1,
)

# convert country boundary to centroid for population table
population_gdf["geometry"] = population_gdf["geometry"].apply(lambda x: x.centroid)

## Create the Vector products

Before creating the Vector products, ensure they do not already exist.

In [ ]:
# ensure tables don't already exist
orgname = dl.auth.Auth().payload["org"]

table_names = [f"{orgname}:country-boundary", f"{orgname}:country-population"]

for table_name in table_names:
    try:
        table = Table.get(table_name)
        table.delete()
    except:
        print(f"{table_name} does not exist!")

Vector allows for creation of custom schemas for each Vector product. The geometry and UUID columns are inherited from the parent model, `models.MultiPolygonBaseModel` and `models.PointBaseModel`, and additional indices can be specified using pydantic Fields.

In [ ]:
class BoundaryModel(models.MultiPolygonBaseModel):
    NAME: str = Field(json_schema_extra={"index": True})
    CONTINENT: str


boundary_table = Table.create(
    product_id="country-boundary",
    name="Country Boundaries",
    description="Not descriptive",
    owners=["org:descarteslabs"],
    model=BoundaryModel,
)


class PopulationModel(models.PointBaseModel):
    NAME: str = Field(json_schema_extra={"index": True})
    POP_EST: float
    GDP_MD_EST: float


population_table = Table.create(
    product_id="country-population",
    name="Country Population",
    description="Country centroids with population estimates.",
    owners=["org:descarteslabs"],
    model=PopulationModel,
)

Creating the product will return a `Table` object.

## Ingest data to the Vector products

Features can be uploaded/ingested to the Vector product by invoking the method `Table.add()`. Adding features will return a `GeoPandas.GeoDataFrame` with UUID attribution.

In [ ]:
# add the boundary data to the boundary table
boundary_gdf = boundary_table.add(boundary_gdf)

# add the population data to the population table
population_gdf = population_table.add(population_gdf)

In [ ]:
population_gdf.head()

In [ ]:
boundary_gdf.head()

## Joins

### TableOptions

Vector products can be filtered/queried by specifying a `property_filter`, `columns`, and `aoi`. In the case of Vector, `property_filter`, `columns`, and `aoi` are collectively referred to as `TableOptions`. Subsequent method calls on the `Table` object will honor these options.
* `property_filter`: Property or column filter for the query. Default is no filter.
* `columns`: A subset of columns to return with each query. Default is all columns will be returned.
* `aoi`: Spatial filter for the query. Default is no spatial filter.


Setting the `TableOptions` can be done during initialization of a `Table` object:

In [ ]:
# setting options on initialization
table1 = Table.get(
    f"{orgname}:country-population",
    property_filter=p.NAME == "Spain",
    columns=["geometry", "NAME", "POP_EST"],
)

df1 = table1.collect()

updated after initialization:

In [ ]:
# updating options after initialization
table2 = Table.get(f"{orgname}:country-population")
table2.options.property_filter = p.NAME == "Spain"
table2.options.columns = ["geometry", "NAME", "POP_EST"]

df2 = table2.collect()

df1 == df2

or overwritten entirely:

In [ ]:
# overwriting the options by explicitly setting the TableOptions options in the collect method
options = TableOptions(
    f"{orgname}:country-population",
    property_filter=p.NAME == "Spain",
    columns=["geometry", "NAME", "POP_EST"],
)
df3 = table1.collect(override_options=options)

df1 == df3

The table options can be reset to default at any point using the `Table.reset_options()` method.

In [ ]:
print(table2.options.columns)
table2.reset_options()
print(table2.options.columns)

### Relational Joins

As seen from the previous examples, calling the `Table.collect()` method will execute a query based on specified `TableOptions`. Upon successful completion, a `GeoPandas.GeoDataFrame` or `Pandas.DataFrame` will be returned. If the `Table` was spatial (i.e. has a geometry column) and the columns option was not set or the geometry column was included in the columns option, a `GeoPandas.GeoDataFrame` will be returned; otherwise, a `Pandas.DataFrame` will be returned. The `DataFrame` will only contain data for the columns specified in the options. To perform a relational join between two Vector products use the `Table.join()` method, which accepts parameters specifying the joining `Table`, join type (`INNER`, `LEFT`, `RIGHT`), and the columns to join on (list of tuples). The `TableOptions` for both the input `Table` and the joining `Table` will be honored upon execution. In the example below, we are constructing an `INNER` join between the boundary table and the population table on the column `NAME`. We have also specified a property filter to only include results where the continent is North America. Since we have not specified a subset of columns for either table, all columns from both tables will be returned. The column naming convention for joined tables follows `PRODUCT_ID.COLUMN`.

In [ ]:
# initialize the boundary table
boundary_table = Table.get(
    f"{orgname}:country-boundary", property_filter=p.CONTINENT == "North America"
)

# initialize the population table
population_table = Table.get(f"{orgname}:country-population")

# join the population table to the boundary table
gdf = boundary_table.join(
    join_table=population_table, join_type="INNER", join_columns=[("NAME", "NAME")]
)

gdf

In the example below, we are executing a `LEFT` join between the boundary table and the population table on the column `NAME`. We have also specified a property filter to on both the input and joining tables. Furthermore, we have specified a subset of columns to include in the joining table, thus only those columns will be joined. The column naming convention for joined tables follows `PRODUCT_ID.COLUMN`.

In [ ]:
# initialize the boundary table
boundary_table = Table.get(
    f"{orgname}:country-boundary", property_filter=p.CONTINENT == "North America"
)

# initialize the population table
population_table = Table.get(
    f"{orgname}:country-population",
    columns=["POP_EST"],
    property_filter=p.POP_EST > 10000000,
)

# join the population table to the boundary table
gdf = boundary_table.join(
    join_table=population_table, join_type="LEFT", join_columns=[("NAME", "NAME")]
)

gdf

### Spatial Joins

Similar to relational joins, two Vector products can be joined spatially using the `Table.sjoin()` method, which accepts parameters specifying the joining `Table` and the join type (`INTERSECTS`, `CONTAINS`, `OVERLAPS`, `WITHIN`). The `TableOptions` for both the input `Table` and the joining `Table` will be honored upon execution. In the example below, we are constructing an `INNER` join between the boundary table and the population table on the column `NAME`. We have also specified a property filter to only include results where the continent is North America. Since we have not specified a subset of columns for either table, all columns from both tables will be returned. The column naming convention for joined tables follows `PRODUCT_ID.COLUMN`.

In [ ]:
# initialize the boundary table
boundary_table = Table.get(
    f"{orgname}:country-boundary", property_filter=p.CONTINENT == "North America"
)

# initialize the population table
population_table = Table.get(
    f"{orgname}:country-population",
    columns=["POP_EST"],
)

# join the population table to the boundary table
gdf = boundary_table.sjoin(join_table=population_table, join_type="CONTAINS")

gdf

In the example above, we are executing a spatial join between the boundary table and the population table where the boundary table contains a point from the population table. We have also specified a property filter to on the input tables. Furthermore, we have specified a subset of columns to include in the joining table, thus only those columns will be joined. The column naming convention for joined tables follows `PRODUCT_ID.COLUMN`.

## Deleting a Vector product

To delete a Vector product, simply invoke the `Table.delete()` method.

In [ ]:
orgname = dl.auth.Auth().payload["org"]

table_names = [f"{orgname}:country-boundary", f"{orgname}:country-population"]

for table_name in table_names:
    try:
        table = Table.get(table_name)
        table.delete()
    except:
        print(f"{table_name} does not exist!")